# Multiclass Prediction in Deep Learning Models

## Read in the Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [3]:
df = pd.read_csv('../Resources/meet_or_beat.csv')

In [4]:
df.shape

(71968, 5)

In [5]:
df.head(3)

,EPS,forecasted_eps,noOfEsts,after_total_returns,before_total_returns
0,2.01,1.67,11.0,0.051444,0.018585
1,0.17,0.19,6.0,0.112955,-0.000510
2,-0.07,0.14,4.0,0.077167,-0.046104


## Preparing the Data

In [6]:
# Generate the categorical outcome variable
df['earnings_outcome'] = np.nan
df.loc[(df['EPS']==df['forecasted_eps']), 'earnings_outcome'] = 'meet'
df.loc[(df['EPS']>df['forecasted_eps']), 'earnings_outcome'] = 'beat'
df.loc[(df['EPS']<df['forecasted_eps']), 'earnings_outcome'] = 'lose'
df.head()

C:\Users\pablo\AppData\Local\Temp\ipykernel_43332\334225973.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'meet' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[(df['EPS']==df['forecasted_eps']), 'earnings_outcome'] = 'meet'


,EPS,forecasted_eps,noOfEsts,after_total_returns,before_total_returns,earnings_outcome
0,2.01,1.67,11.0,0.051444,0.018585,beat
1,0.17,0.19,6.0,0.112955,-0.000510,lose
2,-0.07,0.14,4.0,0.077167,-0.046104,lose
3,0.48,0.51,8.0,-0.006130,-0.004899,lose
4,-0.24,-0.27,9.0,0.089762,-0.025466,beat


In [7]:
# Preview the output variable 
y = df['earnings_outcome']
y

0        beat
1        lose
2        lose
3        lose
4        beat
         ... 
71963    beat
71964    beat
71965    lose
71966    beat
71967    meet
Name: earnings_outcome, Length: 71968, dtype: object

In [8]:
# Encode earnings labels to integers
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
encoded_y

array([0, 1, 1, ..., 1, 0, 2])

In [9]:
# Save the unique number of labels for future use
number_of_classes = len(list(y.drop_duplicates()))
number_of_classes

3

In [10]:
# Convert labeled integers to a Keras `categorical` data type
from tensorflow.keras.utils import to_categorical
y_categorical = to_categorical(encoded_y, num_classes=number_of_classes)
y_categorical


array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [11]:
# Specify X (predictor) variables
X = df[['forecasted_eps',
              'before_total_returns','noOfEsts']]
X.head(3)

,forecasted_eps,before_total_returns,noOfEsts
0,1.67,0.018585,11.0
1,0.19,-0.000510,6.0
2,0.14,-0.046104,4.0


In [12]:
# Split into training and testing windows
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, random_state=1)

In [13]:
# Preview the encoded data we're trying to predict
pd.DataFrame(y_train).head(5)

,0,1,2
0,0.0,1.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,0.0,1.0,0.0
4,1.0,0.0,0.0


In [14]:
# Check for class balance
pd.DataFrame(y_train).sum()

0    32924.0
1    16878.0
2     4174.0
dtype: float64

In [15]:
# Save the count of unique predictor variables for use in model
number_of_predictors = len(X.columns)
len(X.columns)

3

## Building, Fitting and Predicting with Multiple Classes

In [16]:
# Build the neural network layers
model = Sequential()
model.add(Dense(9, input_dim=number_of_predictors, activation='relu'))
model.add(Dense(6, activation='relu'))

C:\Users\pablo\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
# Add the final output layer
# With the # of possible outputs equal to the number classes 
model.add(Dense(number_of_classes, activation='softmax'))

In [18]:
# Compile the model (with multi-class specific parameters)
model.compile(loss="categorical_crossentropy", 
              optimizer= "adam", 
              metrics=['categorical_accuracy'])


In [19]:
# Summarise the structure of the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 9)                   │              36 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │              60 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 3)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 117 (468.00 B)

 Trainable params: 117 (468.00 B)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Fit the model
model.fit(X_train,y_train, 
                    epochs=20,
                    batch_size=1000,
                    shuffle=True)

Epoch 1/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 796us/step - categorical_accuracy: 0.3691 - loss: 1.1370 
Epoch 2/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 624us/step - categorical_accuracy: 0.6114 - loss: 0.9451
Epoch 3/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step - categorical_accuracy: 0.6111 - loss: 0.9477
Epoch 4/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 543us/step - categorical_accuracy: 0.6096 - loss: 0.9016
Epoch 5/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step - categorical_accuracy: 0.6126 - loss: 0.8795
Epoch 6/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 561us/step - categorical_accuracy: 0.6095 - loss: 0.8807
Epoch 7/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step - categorical_accuracy: 0.6115 - loss: 0.8583
Epoch 8/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step - categorical_accuracy: 0.6090 - loss: 0.8795
Epoch 9/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step - categorical_accuracy: 0.6106 - loss: 0.8555
Epoch 10/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step - categorical_accuracy: 0.6089 - loss: 0.8552
Epoch 11/20
54/54 

In [21]:
# Evaluate model on the test data
model.evaluate(X_test,y_test, verbose=2)

563/563 - 0s - 499us/step - categorical_accuracy: 0.6124 - loss: 0.8383


[0.8382949233055115, 0.6123833060264587]

In [22]:
# Save predictions on the test data
predictions = model.predict(X_test)
predictions

563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 434us/step


array([[0.59174377, 0.3301191 , 0.07813708],
       [0.7720915 , 0.15254946, 0.07535906],
       [0.57846683, 0.3614448 , 0.06008842],
       ...,
       [0.6492418 , 0.29108822, 0.05966999],
       [0.57031137, 0.34438515, 0.08530349],
       [0.51338804, 0.40042335, 0.08618858]], dtype=float32)

In [23]:
# Get the most likely prediction for each observation
most_likely = np.argmax(predictions, axis=1)
most_likely

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [24]:
# Convert most likely category back to original labels
most_likely = encoder.inverse_transform((most_likely))
most_likely

array(['beat', 'beat', 'beat', ..., 'beat', 'beat', 'beat'], dtype=object)

In [25]:
# Evaluate prediction balance
pd.DataFrame(most_likely).value_counts()

beat    17722
lose      270
Name: count, dtype: int64